In [1]:
import pandas as pd

train = pd.read_csv('data/따릉이/train.csv')
test = pd.read_csv('data/따릉이/test.csv')

# 전처리

In [2]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [3]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [4]:
# 결측치를 해당 피쳐의 평균값으로 대체하는 코드
# inplace는 권장하지 않음 그래서 살짝 내 입맛대로 바꿨다.
# df[컬럼명] = df[컬럼명].fillna(df[컬럼명].mean())
train['hour_bef_temperature'] = train['hour_bef_temperature'].fillna(train['hour_bef_temperature'].mean())
train['hour_bef_precipitation'] = train['hour_bef_precipitation'].fillna(train['hour_bef_precipitation'].mean())
train['hour_bef_windspeed'] = train['hour_bef_windspeed'].fillna(train['hour_bef_windspeed'].mean())
train['hour_bef_humidity'] = train['hour_bef_humidity'].fillna(train['hour_bef_humidity'].mean())
train['hour_bef_visibility'] = train['hour_bef_visibility'].fillna(train['hour_bef_visibility'].mean())
train['hour_bef_ozone'] = train['hour_bef_ozone'].fillna(train['hour_bef_ozone'].mean())
train['hour_bef_pm10'] = train['hour_bef_pm10'].fillna(train['hour_bef_pm10'].mean())
train['hour_bef_pm2.5'] = train['hour_bef_pm2.5'].fillna(train['hour_bef_pm2.5'].mean())

In [5]:
train.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [6]:
# 결측치를 피쳐의 정보성을 강조하기 위해 보간법으로 대체
# 데이터의 순서는 시간 순서인데 이전 행(직전 시간)과
# 다음 행(직후 시간)의 평균으로 보간하는게 합리적이다
# inplace는 권장하지 않음 그래서 살짝 내 입맛대로 바꿨다.
test = test.interpolate()

In [7]:
test.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
dtype: int64

# 모델링

In [8]:
from sklearn.ensemble import RandomForestRegressor

# model = RandomForestRegressor() # 모델 선언
# mse 평가지표로 훈련 mse로 쓰는 거 곧 사라질거라 squared_error 사용 권장
# criterion{“squared_error”, “absolute_error”, “poisson”}
# default=”squared_error”
model = RandomForestRegressor(criterion = 'squared_error')

In [9]:
train2 = pd.read_csv('data/따릉이/train.csv')
# 결측치 전처리 (보간)
train2 = train2.interpolate()
# count 피쳐 제외
X_train = train2.drop(['count'], axis=1)
# count 피쳐만
Y_train = train2['count']

In [10]:
model.fit(X_train, Y_train)

RandomForestRegressor()

# 튜닝

In [11]:
# 변수의 중요도 파악
# 예측변수를 결정할 때 각 피쳐가 얼마나 중요한 역할을 하는지에 대한 척도
# 중요도가 낮으면 해당 피쳐를 제거하는게 모델 성능 높일 수 있음
model.feature_importances_

array([0.02578906, 0.58525099, 0.18805555, 0.01831456, 0.02540866,
       0.03561158, 0.03445151, 0.03627862, 0.03022006, 0.02061941])

# 피쳐 제거

In [12]:
# 결측치 전처리
train3 = pd.read_csv('data/따릉이/train.csv') 
test3 = pd.read_csv('data/따릉이/test.csv')

train3.interpolate(inplace=True)
test3.fillna(0, inplace=True)

# 모델 훈련
X_train3 = train3.drop(['count'], axis=1)
Y_train3 = train3['count']

model3 = RandomForestRegressor(criterion = 'squared_error')
model3.fit(X_train3, Y_train3)

model3.feature_importances_

array([0.02476285, 0.5903246 , 0.18510642, 0.01741673, 0.02492717,
       0.03723388, 0.03280712, 0.035176  , 0.03230696, 0.01993827])

In [23]:
# drop 경우의 수 대로 X_train 생성
X_train_1 = train3.drop(['count', 'id'], axis=1)
X_train_2 = train3.drop(['count', 'id', 'hour_bef_windspeed'], axis=1)
X_train_3 = train3.drop(['count', 'id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

# drop한 test 셋 생성
test_1 = test3.drop(['id'], axis=1)
test_2 = test3.drop(['id', 'hour_bef_windspeed'], axis=1)
test_3 = test3.drop(['id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

In [24]:
# 각 X_train에 대해 모델 훈련
model_input_var1 = RandomForestRegressor(criterion = 'squared_error')
model_input_var1.fit(X_train_1, Y_train)

model_input_var2 = RandomForestRegressor(criterion = 'squared_error')
model_input_var2.fit(X_train_2, Y_train)

model_input_var3 = RandomForestRegressor(criterion = 'squared_error')
model_input_var3.fit(X_train_3, Y_train)

RandomForestRegressor()

In [25]:
# test 셋 예측
y_pred_1 = model_input_var1.predict(test_1)
y_pred_2 = model_input_var2.predict(test_2)
y_pred_3 = model_input_var3.predict(test_3)

In [26]:
# 파일 저장
submission_1 = pd.read_csv('data/따릉이/submission.csv')
submission_2 = pd.read_csv('data/따릉이/submission.csv')
submission_3 = pd.read_csv('data/따릉이/submission.csv')

submission_1['count'] = y_pred_1
submission_2['count'] = y_pred_2
submission_3['count'] = y_pred_3

submission_1.to_csv('data/따릉이/sub_1.csv', index=False)
submission_2.to_csv('data/따릉이/sub_2.csv', index=False)
submission_3.to_csv('data/따릉이/sub_3.csv', index=False)